In [1]:
import pandas as pd
import torch
import pytorch_lightning as pl
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import torch.utils.data as data
from pytorch_lightning.callbacks import ModelCheckpoint
import numpy as np
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from torchrecsys.datasets import InteractionsDataset, SequenceDataset
from torchrecsys.models import BaseModel
from torchrecsys.task import Ranking
from torchrecsys.layers import BruteForceLayer
import torchrecsys as trs
import matplotlib 
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [2]:
candidates = pd.read_csv("data/candidate_items.csv")
train_purchases = pd.read_csv("data/train_purchases.csv",  parse_dates=['date'])
train_sessions = pd.read_csv("data/train_sessions.csv",  parse_dates=['date'])

test_sessions = pd.read_csv("data/test_leaderboard_sessions.csv",  parse_dates=['date'])
final_test = pd.read_csv("data/test_final_sessions.csv")
all_interactions = pd.concat([train_sessions, train_purchases, test_sessions], ignore_index=True)

features = pd.read_csv("data/item_features.csv")

aux = all_interactions.copy()
n_items = all_interactions.item_id.max()+1

# 1st try 

In [3]:
top_browsed = train_sessions.item_id.value_counts().index.values[:100]

In [4]:
#Score
test_sessions = test_sessions.sort_values(['date']).groupby('session_id')["item_id"].apply(list).to_frame().reset_index()

In [5]:
test_sessions["item_id"] = [top_browsed for _ in range(len(test_sessions))]

In [6]:
df = pd.DataFrame.from_dict(test_sessions[["session_id","item_id"]])
df = df.explode('item_id')
df["rank"] = df.groupby(["session_id"]).cumcount()+1
df.to_csv("predict_toplist.csv", index=False)

In [ ]:
df

# 2nd try 

In [7]:
## SECOND TRY MOST RECENT

In [8]:
top_browsed_recently = train_sessions[(train_sessions['date'] > '2021-02-01') & (train_sessions['date'] < '2021-06-01')].item_id.value_counts().index.values[:100]

In [9]:
#Score
test_sessions = pd.read_csv("data/test_leaderboard_sessions.csv",  parse_dates=['date'])
test_sessions = test_sessions.sort_values(['date']).groupby('session_id')["item_id"].apply(list).to_frame().reset_index()

In [10]:
test_sessions["item_id"] = [top_browsed_recently for _ in range(len(test_sessions))]

In [11]:
df = pd.DataFrame.from_dict(test_sessions[["session_id","item_id"]])
df = df.explode('item_id')
df["rank"] = df.groupby(["session_id"]).cumcount()+1
df.to_csv("predict_toplist_recent.csv", index=False)

In [12]:
df

,session_id,item_id,rank
0,26,26853,1
0,26,19882,2
0,26,21616,3
0,26,8060,4
0,26,7963,5
...,...,...,...
49999,4439757,6627,96
49999,4439757,24948,97
49999,4439757,19270,98
49999,4439757,25320,99


# 3rd try 

In [13]:
## SECOND TRY MOST RECENT

In [14]:
top_bought = train_purchases[(train_purchases['date'] > '2021-02-01') & (train_purchases['date'] < '2021-06-01')].item_id.value_counts().index.values[:100]

In [15]:
#Score

test_sessions = pd.read_csv("data/test_leaderboard_sessions.csv",  parse_dates=['date'])
test_sessions = test_sessions.sort_values(['date']).groupby('session_id')["item_id"].apply(list).to_frame().reset_index()

In [16]:
test_sessions["item_id"] = [top_bought for _ in range(len(test_sessions))]

In [17]:
df = pd.DataFrame.from_dict(test_sessions[["session_id","item_id"]])
df = df.explode('item_id')
df["rank"] = df.groupby(["session_id"]).cumcount()+1
df.to_csv("predict_toplist_bought_recent.csv", index=False)

In [18]:
df

,session_id,item_id,rank
0,26,19882,1
0,26,26853,2
0,26,8060,3
0,26,2915,4
0,26,4193,5
...,...,...,...
49999,4439757,7444,96
49999,4439757,1221,97
49999,4439757,23345,98
49999,4439757,13811,99
